In [1]:
import numpy as np
import sys 
import nmslib 
import time 
import math 
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

A utility function to convert GPS coordinates to Cartesian (X, Y, Z) 

In [2]:
def get_cartesian(lat=None, lon=None):
    lat, lon = np.deg2rad(lat), np.deg2rad(lon)
    R = 6371 # radius of the earth
    x = R * np.cos(lat) * np.cos(lon)
    y = R * np.cos(lat) * np.sin(lon)
    z = R *np.sin(lat)
    return x,y,z

In [ ]:
# Just read the data
gps_data_matrix = np.loadtxt('spacetime.txt')

In [24]:
def convertToCartesian(dataset):    
    cartesian_list = []    
    min_time = dataset[0][2]
    
    for data in dataset:
        lat = data[0]
        lon = data[1]
        time = data[2]
        
        x, y, z = get_cartesian(lat, lon)
        t = time - min_time
        
        cartesian_list.append([x, y, z, t])
    
    cartesian_data = np.asarray(cartesian_list)    
    return cartesian_data

In [25]:
data_matrix = convertToCartesian(gps_data_matrix)
print (data_matrix.shape)

(555079, 4)


In [26]:
M = 15
efC = 100

num_threads = 4
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}
space_name='l2'

index = nmslib.init(method='hnsw', space=space_name, data_type=nmslib.DataType.DENSE_VECTOR) 
index.addDataPointBatch(data_matrix)

555079

In [27]:
#Create the index

start = time.time()
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC}
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))

Index-time parameters {'M': 15, 'indexThreadQty': 4, 'efConstruction': 100}
Indexing time = 30.594989


In [29]:
index.saveIndex('spacetime_index.bin', save_data=False)